In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



In [2]:
!pip install ../input/talib-package/talib_binary-0.4.19-cp37-cp37m-manylinux1_x86_64.whl
import talib as ta 

Processing /kaggle/input/talib-package/talib_binary-0.4.19-cp37-cp37m-manylinux1_x86_64.whl


In [3]:
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from lightgbm import early_stopping
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, MinMaxScaler
import missingno as msno

from pylab import rcParams

from tqdm import tqdm

# import ta
from talib import abstract


import time
import gc
import sys

from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GroupKFold
from sklearn.metrics import r2_score, mean_squared_error


import seaborn as sns
import plotly.graph_objects as go

from decimal import ROUND_HALF_UP, Decimal

import optuna
from optuna.integration import LightGBMPruningCallback

import joblib

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
# sys.path.insert(0, '../input/jpx-local-api')
# from local_api import local_api

In [4]:
train = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv')

# using supplement data as test data
supp_data = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv')

# stock code
stock_list = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv')
main_list = stock_list.set_index('SecuritiesCode').loc[np.sort(train['SecuritiesCode'].unique())]

# add stock type to data
train = pd.merge(train, main_list['33SectorCode'].astype('category'), on='SecuritiesCode').sort_values(['Date','SecuritiesCode']).astype({'SecuritiesCode':'category'})

supp_data = pd.merge(supp_data, main_list['33SectorCode'].astype('category'), on='SecuritiesCode').sort_values(['Date','SecuritiesCode']).astype({'SecuritiesCode':'category'})

train_with_supp = pd.concat([train, supp_data]).reset_index(drop=True)
# train_with_supp = train.copy()
train_with_supp
# train

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,33SectorCode
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,50
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324,50
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154,50
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053,50
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2602407,20220624_9990,2022-06-24,9990,576.0,576.0,563.0,564.0,24200,1.0,NaN,False,0.027073,6100
2602408,20220624_9991,2022-06-24,9991,810.0,815.0,804.0,815.0,8700,1.0,NaN,False,0.001220,6050
2602409,20220624_9993,2022-06-24,9993,1548.0,1548.0,1497.0,1497.0,12600,1.0,NaN,False,0.001329,6100
2602410,20220624_9994,2022-06-24,9994,2507.0,2527.0,2498.0,2527.0,7300,1.0,NaN,False,0.003185,6100


In [5]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

In [6]:
def generate_adjusted_close(df):
    """
    Args:
        df (pd.DataFrame)  : stock_price for a single SecuritiesCode
    Returns:
        df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
    """
    # sort data to generate CumulativeAdjustmentFactor
    df = df.sort_values("Date", ascending=False)
    # generate CumulativeAdjustmentFactor
    df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
    # generate AdjustedClose
    df.loc[:, "AdjustedClose"] = (
        df["CumulativeAdjustmentFactor"] * df["Close"]
    ).map(lambda x: float(
        Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
    ))
    # reverse order
    df = df.sort_values("Date")
    # to fill AdjustedClose, replace 0 into np.nan
    df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
    # forward fill AdjustedClose
    df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()

    df = df.drop(columns=['CumulativeAdjustmentFactor'])
    
    return df

In [7]:
def get_ta_features(df, inf=None, train=True):
    """
    Get technical features from TA-Lib
    ref : https://www.kaggle.com/code/daosword/jpx-pytorch-neural-network-with-ta-lib-features
    """
    if train:
        op = df['Open']
        hi = df['High']
        lo = df['Low']
        cl = df['AdjustedClose']  #df['Close']
        vo = df['Volume']

    #     # Overlap Studies
    #     df['BBANDS_upper'], df['BBANDS_middle'], df['BBANDS_lower'] = ta.BBANDS(cl, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    #     df['DEMA'] = ta.DEMA(cl, timeperiod=30)
        for period in [7,15,30]:
            df['EMA{}'.format(period)] = cl.ewm(span=period,  adjust=False).mean().values/cl
            df['return{}'.format(period)] = cl.pct_change(period)
            df['volatility{}'.format(period)] = np.log(cl).diff().rolling(period).std()
            
            


        # df['EMA7'] = ta.EMA(cl, 7)/cl
        # df['EMA15'] = ta.EMA(cl, 15)/cl
        # df['EMA30'] = ta.EMA(cl, 30)/cl
        # df['EMA90'] = ta.EMA(cl, 90)/cl

    #     df['HT_TRENDLINE'] = ta.HT_TRENDLINE(cl)
    #     df['KAMA'] = ta.KAMA(cl, timeperiod=30)
    #     df['MA'] = ta.MA(cl, timeperiod=30, matype=0)
    #     df['MIDPOINT'] = ta.MIDPOINT(cl, timeperiod=14)
    #     df['SAR'] = ta.SAR(hi, lo, acceleration=0, maximum=0)
    #     df['SAREXT'] = ta.SAREXT(hi, lo, startvalue=0, offsetonreverse=0, accelerationinitlong=0, accelerationlong=0, accelerationmaxlong=0, accelerationinitshort=0, accelerationshort=0, accelerationmaxshort=0)
    #     df['SMA'] = ta.SMA(cl, timeperiod=30)
    #     df['T3'] = ta.T3(df['Close'], timeperiod=5, vfactor=0)
    #     df['TEMA'] = ta.TEMA(df['Close'], timeperiod=30)
    #     df['TRIMA'] = ta.TRIMA(df['Close'], timeperiod=30)
    #     df['WMA'] = ta.WMA(df['Close'], timeperiod=30)

        # Momentum Indicators
    #     df['ADX'] = ta.ADX(hi, lo, cl, timeperiod=14)
    #     df['ADXR'] = ta.ADXR(hi, lo, cl, timeperiod=14)
    #     df['APO'] = ta.APO(cl, fastperiod=12, slowperiod=26, matype=0)
    #     df['AROON_down'], df['AROON_up'] = ta.AROON(hi, lo, timeperiod=14)
    #     df['AROONOSC'] = ta.AROONOSC(hi, lo, timeperiod=14)
    #     df['BOP'] = ta.BOP(op, hi, lo, cl)
    #     df['CCI'] = ta.CCI(hi, lo, cl, timeperiod=14)
    #     df['DX'] = ta.DX(hi, lo, cl, timeperiod=14)
    #     df['MACD_macd'], df['MACD_macdsignal'], df['MACD_macdhist'] = ta.MACD(cl, fastperiod=12, slowperiod=26, signalperiod=9)
    #     df['MFI'] = ta.MFI(hi, lo, cl, vo, timeperiod=14)
    #     df['MINUS_DI'] = ta.MINUS_DI(hi, lo, cl, timeperiod=14)
    #     df['MINUS_DM'] = ta.MINUS_DM(hi, lo, timeperiod=14)
        df['MOM'] = ta.MOM(cl, timeperiod=10)
    #     df['PLUS_DI'] = ta.PLUS_DI(hi, lo, cl, timeperiod=14)
    #     df['PLUS_DM'] = ta.PLUS_DM(hi, lo, timeperiod=14)
        df['RSI'] = ta.RSI(cl, timeperiod=14)
        df['STOCH_slowk'], df['STOCH_slowd'] = ta.STOCH(hi, lo, cl, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
        df['STOCHF_fastk'], df['STOCHF_fastd'] = ta.STOCHF(hi, lo, cl, fastk_period=5, fastd_period=3, fastd_matype=0)
        df['STOCHRSI_fastk'], df['STOCHRSI_fastd'] = ta.STOCHRSI(cl, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
    #     df['TRIX'] = ta.TRIX(cl, timeperiod=30)
    #     df['ULTOSC'] = ta.ULTOSC(hi, lo, cl, timeperiod1=7, timeperiod2=14, timeperiod3=28)
    #     df['WILLR'] = ta.WILLR(hi, lo, cl, timeperiod=14)

        # Volume Indicators
    #     df['AD'] = ta.AD(hi, lo, cl, vo)
    #     df['ADOSC'] = ta.ADOSC(hi, lo, cl, vo, fastperiod=3, slowperiod=10)
    #     df['OBV'] = ta.OBV(cl, vo)

        # Volatility Indicators
        df['ATR'] = ta.ATR(hi, lo, cl, timeperiod=14)
        df['NATR'] = ta.NATR(hi, lo, cl, timeperiod=14)
        df['TRANGE'] = ta.TRANGE(hi, lo, cl)

        # Cycle Indicators
    #     df['HT_DCPERIOD'] = ta.HT_DCPERIOD(cl)
    #     df['HT_DCPHASE'] = ta.HT_DCPHASE(cl)
    #     df['HT_PHASOR_inphase'], df['HT_PHASOR_quadrature'] = ta.HT_PHASOR(cl)
    #     df['HT_SINE_sine'], df['HT_SINE_leadsine'] = ta.HT_SINE(cl)
    #     df['HT_TRENDMODE'] = ta.HT_TRENDMODE(cl)

        # Statistic Functions
    #     df['BETA'] = ta.BETA(hi, lo, timeperiod=5)
    #     df['CORREL'] = ta.CORREL(hi, lo, timeperiod=30)
    #     df['LINEARREG'] = ta.LINEARREG(cl, timeperiod=14) - cl
    #     df['LINEARREG_ANGLE'] = ta.LINEARREG_ANGLE(cl, timeperiod=14)
    #     df['LINEARREG_INTERCEPT'] = ta.LINEARREG_INTERCEPT(cl, timeperiod=14) - cl
    #     df['LINEARREG_SLOPE'] = ta.LINEARREG_SLOPE(cl, timeperiod=14)
        df['STDDEV'] = ta.STDDEV(cl, timeperiod=5, nbdev=1)   
    
    
    else: #inference 
        op = inf['Open']
        hi = inf['High']
        lo = inf['Low']
        cl = inf['Close']
        vo = inf['Volume']
        
            #     # Overlap Studies
    #     df['BBANDS_upper'], df['BBANDS_middle'], df['BBANDS_lower'] = ta.BBANDS(cl, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    #     df['DEMA'] = ta.DEMA(cl, timeperiod=30)

        # df['EMA7'] = cl.ewm(span=7, min_periods=3, adjust=False).mean().iloc[-1]/cl
        # df['EMA15'] = cl.ewm(span=15, min_periods=7, adjust=False).mean().iloc[-1]/cl
        # df['EMA30'] = cl.ewm(span=30, min_periods=15, adjust=False).mean().iloc[-1]/cl
        # df['EMA90'] = cl.ewm(span=90, min_periods=30, adjust=False).mean().iloc[-1]/cl

        for period in [7,15,30]:
            df['EMA{}'.format(period)] = (cl.ewm(span=period,  adjust=False).mean().iloc[-1]/cl).iloc[-1]
            df['return{}'.format(period)] = (cl.pct_change(period)).iloc[-1]
            df['volatility{}'.format(period)] = (np.log(cl).diff().rolling(period).std()).iloc[-1]

    #     df['HT_TRENDLINE'] = ta.HT_TRENDLINE(cl)
    #     df['KAMA'] = ta.KAMA(cl, timeperiod=30)
    #     df['MA'] = ta.MA(cl, timeperiod=30, matype=0)
    #     df['MIDPOINT'] = ta.MIDPOINT(cl, timeperiod=14)
    #     df['SAR'] = ta.SAR(hi, lo, acceleration=0, maximum=0)
    #     df['SAREXT'] = ta.SAREXT(hi, lo, startvalue=0, offsetonreverse=0, accelerationinitlong=0, accelerationlong=0, accelerationmaxlong=0, accelerationinitshort=0, accelerationshort=0, accelerationmaxshort=0)
    #     df['SMA'] = ta.SMA(cl, timeperiod=30)
    #     df['T3'] = ta.T3(df['Close'], timeperiod=5, vfactor=0)
    #     df['TEMA'] = ta.TEMA(df['Close'], timeperiod=30)
    #     df['TRIMA'] = ta.TRIMA(df['Close'], timeperiod=30)
    #     df['WMA'] = ta.WMA(df['Close'], timeperiod=30)

        # Momentum Indicators
    #     df['ADX'] = ta.ADX(hi, lo, cl, timeperiod=14)
    #     df['ADXR'] = ta.ADXR(hi, lo, cl, timeperiod=14)
    #     df['APO'] = ta.APO(cl, fastperiod=12, slowperiod=26, matype=0)
    #     df['AROON_down'], df['AROON_up'] = ta.AROON(hi, lo, timeperiod=14)
    #     df['AROONOSC'] = ta.AROONOSC(hi, lo, timeperiod=14)
    #     df['BOP'] = ta.BOP(op, hi, lo, cl)
    #     df['CCI'] = ta.CCI(hi, lo, cl, timeperiod=14)
    #     df['DX'] = ta.DX(hi, lo, cl, timeperiod=14)
    #     df['MACD_macd'], df['MACD_macdsignal'], df['MACD_macdhist'] = ta.MACD(cl, fastperiod=12, slowperiod=26, signalperiod=9)
    #     df['MFI'] = ta.MFI(hi, lo, cl, vo, timeperiod=14)
    #     df['MINUS_DI'] = ta.MINUS_DI(hi, lo, cl, timeperiod=14)
    #     df['MINUS_DM'] = ta.MINUS_DM(hi, lo, timeperiod=14)
        df['MOM'] = ta.MOM(cl, timeperiod=10).iloc[-1]
    #     df['PLUS_DI'] = ta.PLUS_DI(hi, lo, cl, timeperiod=14)
    #     df['PLUS_DM'] = ta.PLUS_DM(hi, lo, timeperiod=14)
        df['RSI'] = ta.RSI(cl, timeperiod=14).iloc[-1]
        df['STOCH_slowk'], df['STOCH_slowd'] = (ta.STOCH(hi, lo, cl, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)[0].iloc[-1],
                                                ta.STOCH(hi, lo, cl, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)[1].iloc[-1])
        df['STOCHF_fastk'], df['STOCHF_fastd'] = (ta.STOCHF(hi, lo, cl, fastk_period=5, fastd_period=3, fastd_matype=0)[0].iloc[-1],
                                                  ta.STOCHF(hi, lo, cl, fastk_period=5, fastd_period=3, fastd_matype=0)[1].iloc[-1])
        df['STOCHRSI_fastk'], df['STOCHRSI_fastd'] = (ta.STOCHRSI(cl, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)[0].iloc[-1],
                                                      ta.STOCHRSI(cl, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)[1].iloc[-1])
    #     df['TRIX'] = ta.TRIX(cl, timeperiod=30)
    #     df['ULTOSC'] = ta.ULTOSC(hi, lo, cl, timeperiod1=7, timeperiod2=14, timeperiod3=28)
    #     df['WILLR'] = ta.WILLR(hi, lo, cl, timeperiod=14)

        # Volume Indicators
    #     df['AD'] = ta.AD(hi, lo, cl, vo)
    #     df['ADOSC'] = ta.ADOSC(hi, lo, cl, vo, fastperiod=3, slowperiod=10)
    #     df['OBV'] = ta.OBV(cl, vo)

        # Volatility Indicators
        df['ATR'] = ta.ATR(hi, lo, cl, timeperiod=14).iloc[-1]
        df['NATR'] = ta.NATR(hi, lo, cl, timeperiod=14).iloc[-1]
        df['TRANGE'] = ta.TRANGE(hi, lo, cl).iloc[-1]

        # Cycle Indicators
    #     df['HT_DCPERIOD'] = ta.HT_DCPERIOD(cl)
    #     df['HT_DCPHASE'] = ta.HT_DCPHASE(cl)
    #     df['HT_PHASOR_inphase'], df['HT_PHASOR_quadrature'] = ta.HT_PHASOR(cl)
    #     df['HT_SINE_sine'], df['HT_SINE_leadsine'] = ta.HT_SINE(cl)
    #     df['HT_TRENDMODE'] = ta.HT_TRENDMODE(cl)

        # Statistic Functions
    #     df['BETA'] = ta.BETA(hi, lo, timeperiod=5)
    #     df['CORREL'] = ta.CORREL(hi, lo, timeperiod=30)
    #     df['LINEARREG'] = ta.LINEARREG(cl, timeperiod=14) - cl
    #     df['LINEARREG_ANGLE'] = ta.LINEARREG_ANGLE(cl, timeperiod=14)
    #     df['LINEARREG_INTERCEPT'] = ta.LINEARREG_INTERCEPT(cl, timeperiod=14) - cl
    #     df['LINEARREG_SLOPE'] = ta.LINEARREG_SLOPE(cl, timeperiod=14)
        df['STDDEV'] = ta.STDDEV(cl, timeperiod=5, nbdev=1).iloc[-1]   
    
    return df

In [8]:
def divideSecurities(df):
    sec_list = []
#     print('Divide securities individually..')
    for code in np.sort(df.SecuritiesCode.unique()):
        sec_list.append(df.loc[df.SecuritiesCode == code, :].reset_index(drop=True))
    return sec_list

sec_list = divideSecurities(train)

In [9]:
def add_features_train(sec_list):
    df_list = []
    for df in tqdm(sec_list):

        # 결측치 채워넣기
        df = df.fillna(method='pad')

        # shadows
        df['upper_shadow'] = df['High'] - np.maximum(df['Open'], df['Close'])
        df['lower_shadow'] = np.minimum(df['Open'], df['Close']) - df['Low']
        
        

        # adjusted close
        df = generate_adjusted_close(df)

        ## Rolling features ##
        
        # lagged features
        # 날짜 단위이므로 7일전, 30일전, 180일전, 360일전 
        # lagged close, target (target 은 정확히 무엇? return인가)
        
        
        # df = df.interpolate()
        
        
        
        # All indicators in ta
        df = get_ta_features(df.copy())
        # try:
        #     df = get_ta_features(df.copy())
        # except:
        #     print(f"error in SecuritiesCode: {df['SecuritiesCode'][0]}")
        #     display(df)
        #     continue
                
        # not add pattern recognition - feature importance 가 거의 0임. 
#         for indicator in ta.get_function_groups()['Pattern Recognition']:
#             df[str(indicator)] = getattr(ta,str(indicator))(df.Open, df.High, df.Low, df.Close)


        ####  For test!! df['STOCHF_fastd'] df['STOCHRSI_fastk'] df['ATR']
        ####  open high low close 등 제거
        df = df.drop(columns=['STOCHF_fastd', 'STOCHRSI_fastk', 'STOCH_slowk', 'ATR', 
                                'Open', 'High', 'Low'])
    #     df = df.drop(columns=['upper_shadow', 'Low', 'TRANGE', 'High',
    #    'STOCHRSI_fastk', 'STOCHRSI_fastd', 'Open'])
        # fill ema features by backward -- 이렇게 채워진 것은 false data 이므로 일단 test 해보고 없애는 것을 검토하자.
        # df = df.fillna(method='ffill')
        # df = df.interpolate()
        
        # volatility
        
        df_list.append(df)
        
        del df
        
    # gc.collect()

    df_feature_added = pd.concat(df_list).sort_values(['Date','SecuritiesCode'])
    
    dfc = df_feature_added.drop(columns=['RowId', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag'])
    dfc = dfc.dropna()   # 60000 nan rows deleted
    
    y = dfc.set_index(['Date']).loc[:, 'Target']

    dfc = dfc.set_index(['Date']).drop(columns=['Target', 'Close'])

    
    
    
    return dfc, y




In [10]:
df_added, y = add_features_train(sec_list)
# df_added = df_added[(df_added['Date'] >= '2021-06-03') & (df_added['Date'] <= '2021-12-03')]
# df_added = df_added[df_added['Date'] >= '2021-05-27']
# df_added.to_csv('train_with_supp_feature_added.csv', index=False)

# load data
# df_added = pd.read_csv('/kaggle/input/train-with-supp-feature-added-v1-all-cdl/train_with_supp_feature_added.csv')

# df_added

100%|██████████| 2000/2000 [00:53<00:00, 37.54it/s]


In [11]:
def add_features_infer(input_df, close_df): #input df 는 price 데이터
    
    df_list = []
    
    print('Divide input securities...')
    sec_list = divideSecurities(input_df)
    print('Divide close_df securities...')
    close_list = divideSecurities(close_df) # for rolling features
    print('='*10 + 'feature adding' + '='*10)
    
    assert len(sec_list) == len(close_list)
    
    for i in range(len(sec_list)):
        
        
        close = close_list[i] #.loc[close_df.SecuritiesCode == code, :].fillna(method='ffill')
        df = sec_list[i]
#         # test data의 open, high, low, close 중 nan 있으면 이전 값에서 가져와 채움
        if df.loc[:, ['Open', 'High', 'Low', 'Close']].isna().any().any():
            print('NaN detected..')
            display(df)
#             display(close)
# #             print(close.iloc[-2]['Open', 'High', 'Low', 'Close'].values())
            df.loc[:, ['Open', 'High', 'Low', 'Close', 'Volume']] = close.loc[close['Date'] == close.iloc[-2]['Date'], ['Open', 'High', 'Low', 'Close', 'Volume']].values 
            print('NaN filled...')
            display(df)
        

        
        # shadows
        df['upper_shadow'] = df['High'] - np.maximum(df['Open'], df['Close'])
        df['lower_shadow'] = np.minimum(df['Open'], df['Close']) - df['Low']

        # lagged features
        # 날짜 단위이므로 7일전, 30일전, 180일전, 360일전 
        # lagged close, target (target 은 정확히 무엇? return인가)
        
        # adjusted close
        df = generate_adjusted_close(df)
        
        ## Rolling features ##
        # TA-lib features
        df = get_ta_features(df, close, train=False)
        
        ####  open high low close 등 제거
        df = df.drop(columns=['STOCHF_fastd', 'STOCHRSI_fastk', 'STOCH_slowk', 'NATR', 'ATR'])
        
#         for indicator in ta.get_function_groups()['Pattern Recognition']:
#             df[str(indicator)] = getattr(ta,str(indicator))(df.Open, df.High, df.Low, df.Close)


        # volatility
        
        df_list.append(df)
        del df
    
    df_feature_added = pd.concat(df_list)
    
    return df_feature_added



In [12]:
def preprocess_inference(df, trained_scalers: list):
    ordinal = trained_scalers[0]
    stdsc = trained_scalers[1]
    
      
    # remove columns - Date removed temporarily
    dfc = df.drop(columns=['RowId', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag'])
    
    
    target = ['Target']
    ord_features = ['SecuritiesCode'] 
    scaled_features = [i for i in df.columns if i not in ['Date', 'RowId', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag',
                                                         'Target', 'SecuritiesCode']]
#     print('scaled_features:', scaled_features)
    date_code_ord = ordinal.transform(dfc.loc[:,ord_features])
    scaled = stdsc.transform(dfc.loc[:,scaled_features])
    


    dfc_scaled = pd.concat([df['Date'].reset_index(drop=True),
                            pd.DataFrame(date_code_ord, columns=ord_features),
                            pd.DataFrame(scaled, columns=scaled_features)], axis=1)

    dfc_scaled = dfc_scaled.set_index(['Date']).drop(columns=['Close'])

    

#     y = dfc.loc[:, ['Target']]
    
    return dfc_scaled


# X_test_scaled = preprocess_train(df_added, trained_scalers)

# X_test_scaled

In [13]:
# scaled
stdsc = StandardScaler()
scaled_features = [i for i in df_added.columns if i not in ['SecuritiesCode', '33SectorCode']]
scaled = stdsc.fit_transform(df_added.loc[:,scaled_features])
df_added_scaled =  pd.concat([
                            df_added.loc[:,['SecuritiesCode','33SectorCode']].reset_index(),
                            pd.DataFrame(scaled, columns=scaled_features)
                            ], axis=1).set_index('Date')

In [14]:
# maxsr_study = joblib.load('../models/lgbm_fin_maxSR_10trial_stationary_scaled_train130_test60_fold8.pkl')
maxsr_params = {
    'n_estimators': 500,
    'learning_rate': 0.001434850342256775,
    'num_leaves': 226,
    'max_depth': 5,
    'min_data_in_leaf': 4900,
    'lambda_l1': 0.5144342052290969,
    'lambda_l2': 0.0016086108752855683,
    'bagging_fraction': 0.9,
    'feature_fraction': 0.7,
    'max_bin': 64
}

In [15]:
# base model - lgbm - only 130 days for training
# test - time-series cv 5-fold ensemble
class EnsembleModel:
    def __init__(self, models: list):
        self.models = models
    
    def predict(self, x):
        predicted = np.zeros((len(x), len(self.models)))
        for i, model in enumerate(self.models):
            predicted[:, i] = model.predict(x)
        return np.sum(predicted, axis=1) / len(self.models)
        
def group_timeseriesCV(grid, train_size=210, test_size=60, fold=5, total=False):  # overlap last test and train
    # last cv should be end the previous day of eval start  -  마지막 cv eval이 test data 직전에 끝나게 설정
    # cv_size = train_size + test_size
    if total:
        cv_train = grid[:-test_size]
        cv_test = grid[-test_size:]
        yield cv_train, cv_test
    else:
        for i in range(fold):
            cv_train = grid[-(train_size + test_size + train_size*(fold-i-1)):-(test_size + train_size*(fold-i-1))]
            test_end = -train_size*(fold-i-1) if i < fold-1 else None
            cv_test = grid[-(test_size + train_size*(fold-i-1)): test_end]              
            yield cv_train, cv_test


# cates = ['SecuritiesCode', '33SectorCode']  # categorical columns

grid = df_added.index.unique() 
# print(min(grid), max(grid))
# tscv = TimeSeriesSplit(n_splits=7, test_size=60)

k = 0
feat_importance = pd.DataFrame()
models = []
cv_score = []
r2_list = []

for train_idx, eval_idx in group_timeseriesCV(grid, 130, 60, fold=8): #  #

# for train_idx, eval_idx in tscv.split(grid):
    # train_idx = grid[train_idx]
    # eval_idx = grid[eval_idx]

    k += 1
    t0 = time.time()
    print(f'========Training fold {k}========')
   
    print('training size:', len(train_idx), '  test size:', len(eval_idx))
    
    print("Train Date range: {} to {}".format(train_idx.min(),train_idx.max()))
    print("Valid Date range: {} to {}".format(eval_idx.min(),eval_idx.max()))
    

    X, y = df_added_scaled.copy(), y.copy()
    # X, y = X_lgb.copy(), y.copy()

    X_train, X_test = X.loc[train_idx].reset_index(drop=True), X.loc[eval_idx].reset_index(drop=True)
    y_train, y_test = y.loc[train_idx].reset_index(drop=True), y.loc[eval_idx].reset_index(drop=True)

    # handpick_params = {
    #     'n_estimators': 2000,
    #     'learning_rate': 0.001,
    #     'num_leaves': 109,
    #     'max_depth': 10,
    #     'min_data_in_leaf': 2400,
    #     'bagging_fraction': 0.5,
    #     'feature_fraction': 0.2,
    #     'max_bin': 106
    #         }
    # optuna_params = {
    #     'lambda_l1': 0.00037684382460392624,
    #     'lambda_l2': 0.00751489594500413,
    #         }

    # lgb = LGBMRegressor(**handpick_params, **optuna_params, seed=2022)#, categorical_feature='name:SecuritiesCode,33SectorCode')
    
    lgb = LGBMRegressor(**maxsr_params, seed=2022)
    lgb.fit(
        X_train, 
        y_train,
        eval_set = [(X_test, y_test)],
        eval_metric='rmse',
        callbacks=[
                early_stopping(stopping_rounds=20)
            ]
        )
    
    models.append(lgb)

    # feat_importance["Importance_Fold"+str(k)]=lgb.feature_importances_
    # feat_importance.set_index(X_train.columns, inplace=True)

    # y_pred = lgb.predict(X_test)
    y_pred = lgb.predict(X_test)
    
    # Sharpe Ratio 
    df_sharpe = pd.DataFrame([])
    df_sharpe['Date'] = X.loc[eval_idx].index
    df_sharpe['predict'] = y_pred
    df_sharpe['Target'] = y_test
    df_sharpe['Rank'] = (df_sharpe.groupby('Date')['predict'].rank(method='first', ascending=False)-1).astype(int)
#         display(X_test.groupby('Date')['Rank'].min())
#         display(X_test.groupby('Date')['Rank'].max())
#         display(X_test)
    eval_score = calc_spread_return_sharpe(df_sharpe)
    cv_score.append(eval_score)

    print(f'Fold {k} evaluated in {time.time() - t0 :.3f}s')
    print(f'Fold {k} sharpe ratio score: {eval_score :.6f}')
    
    r2_list.append(r2_score(y_test.values, y_pred))
    print('R2-score: {:.6f}'.format(r2_score(y_test.values, y_pred)))

print(f'cv mean sharpe ratio score: {np.mean(cv_score) :.6f}')
print(f'cv STD sharpe ratio score: {np.std(cv_score) :.6f}')
print(f'cv mean r2 score: {np.mean(r2_list) :.6f}')
# score_type = 'scaled_6fold_basicfeatonly'
# if score_type in score_board.keys():
#     raise ValueError('key already exists')
# else:
#     score_board[score_type] = np.mean(cv_score)

# lgb = LGBMRegressor(device='GPU',**params).fit(X_scaled, y)



# feat_importance['avg'] = feat_importance.mean(axis=1)
# feat_importance = feat_importance.sort_values(by='avg',ascending=True)
# display(feat_importance)



========Training fold 1========
training size: 130   test size: 60
Train Date range: 2017-06-02 to 2017-12-08
Valid Date range: 2017-12-11 to 2018-03-09
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] lambda_l1 is set=0.5144342052290969, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5144342052290969
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] lambda_l2 is set=0.0016086108752855683, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0016086108752855683
[LightGBM] [Warning] min_data_in_leaf is set=4900, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4900
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's rmse: 0.0234228	valid_0's l2: 0.000548627
Fold 1 evaluated in 11.741s
Fold 1 sharp

In [16]:
# Ensemble the cv models
ensemble = EnsembleModel(models)

### LGBM Fitting result log

1. ~30 window, tscv, 7-fold, stationary+scaled  
cv mean sharpe ratio score: 0.069140  
cv STD sharpe ratio score: 0.094025  
cv mean r2 score: -0.004279   


1. <mark> **[Selected for Optuna HP tuning]** ~30 window, 130 days, 8-fold, stationary features + scaled    
cv mean sharpe ratio score: 0.123441  
cv STD sharpe ratio score: 0.109005  
cv mean r2 score: -0.003533</mark>

1. <mark> **[Optuna HP tuned]** ~30 window, 130 days, 8-fold, stationary features + scaled    
cv mean sharpe ratio score: 0.139032  
cv STD sharpe ratio score: 0.146264  
cv mean r2 score: -0.003247  


1. ~30 window, 130 days, 8-fold, features scaled except categorical   
cv mean sharpe ratio score: 0.128036  
cv STD sharpe ratio score: 0.144154  
cv mean r2 score: -0.003573  

1. ~30 window, all days + 2022-03~05 test with restored features and ensemble model(~2022-03)  
cv mean sharpe ratio score: 0.022831  
cv STD sharpe ratio score: 0.000000  
cv mean r2 score: -0.000224


1. ~30 window, 130 days, + test period all (2022-05), removed features restored , ensemble model(~2022-03)  
cv mean sharpe ratio score: 0.241770  
cv STD sharpe ratio score: 0.155396  
cv mean r2 score: -0.001687  

1. ~30 window, 130 days, + test period, removed features restored , ensemble model   
cv mean sharpe ratio score: 0.232070   
cv STD sharpe ratio score: 0.245231   
cv mean r2 score: -0.003154  

1. ~30 window, 130 days, + test period, removed features restored  
cv mean sharpe ratio score: 0.089397  
cv STD sharpe ratio score: 0.155554  
cv mean r2 score: -0.003571  

1. ~30 window, 130 days + test period  
cv mean sharpe ratio score: -0.012974  
cv STD sharpe ratio score: 0.172770  
cv mean r2 score: -0.003650  



1. ~30 window, 210 days train    
cv mean sharpe ratio score: 0.004027  
cv STD sharpe ratio score: 0.116113   
cv mean r2 score: -0.012231  

2. ~30 window, total train (1112 days)  
cv mean sharpe ratio score: -0.014474  
cv STD sharpe ratio score: 0.000000  
cv mean r2 score: -0.002762


3. ~30 window, 130 days train (cv: 8)  
cv mean sharpe ratio score: 0.060078  
cv STD sharpe ratio score: 0.152693  
cv mean r2 score: -0.003789  


2. ~90 window, 210 days train  
cv mean sharpe ratio score: 0.024323  
mean R2-score: -0.012523      
 
3. ~90 window, 350 days train  
cv mean sharpe ratio score: 0.055422  
cv STD sharpe ratio score: 0.033707  
cv mean r2 score: -0.008804  

In [17]:
# # feat importance plot
# feat_importance['avg'] = feat_importance.mean(axis=1)
# feat_importance = feat_importance.sort_values(by='avg',ascending=True)
# pal=sns.color_palette("plasma_r", 40).as_hex()[2:]


# fig=go.Figure()
# for i in range(len(feat_importance.index)):
#     fig.add_shape(dict(type="line", y0=i, y1=i, x0=0, x1=feat_importance['avg'][i], 
#                        line_color=pal[::-1][i],opacity=0.7,line_width=4))
# fig.add_trace(go.Scatter(x=feat_importance['avg'], y=feat_importance.index, mode='markers', 
#                          marker_color=pal[::-1], marker_size=8,
#                          hovertemplate='%{y} Importance = %{x:.0f}<extra></extra>'))
# fig.show()

In [18]:
# # only using average cv score

# def objective(trial, df, y, train_size, test_size, fold):  # X_scaled, X_scaled_val, y, y_val
    
#     global count
#     print('')
#     print('========= Optuna HyperParameter Tuning trial {} =========='.format(count))
#     print('')
#     param_grid = {
#         # "device_type": trial.suggest_categorical("device_type", ['gpu']),
#         'objective': 'regression',
#         "seed": 2022,

#         "n_estimators": trial.suggest_categorical("n_estimators", [500, 1000, 2000, 4000, 5000, 10000]),
#         "learning_rate": trial.suggest_loguniform("learning_rate", 5e-4, 0.5),
#         "num_leaves": trial.suggest_int("num_leaves", 2, 256),
#         "max_depth": trial.suggest_int("max_depth", 2, 12),
#         "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
#         'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
#         'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
# #         "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
#         "bagging_fraction": trial.suggest_float(
#             "bagging_fraction", 0.2, 0.9, step=0.1
#         ),
# #         "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
#         "feature_fraction": trial.suggest_float(
#             "feature_fraction", 0.2, 0.9, step=0.1
#         ),
        
#         'max_bin': trial.suggest_int('max_bin', 15, 256)
        
#     }

#     # tscv = TimeSeriesSplit(n_splits=k, test_size=60)

#     cv_score = []
#     cv_sharpe = []
#     grid = df.index.unique() 
#     feat_importance = pd.DataFrame([])
    
#     for i, (train_idx, eval_idx) in enumerate(group_timeseriesCV(grid, train_size, test_size, fold)):
#     # for i, (train_idx, eval_idx) in enumerate(tscv.split(grid)):
#     #     train_idx = grid[train_idx]
#     #     eval_idx = grid[eval_idx]

        
#         t0 = time.time()
#         print(f'========Training fold {i+1}========')

#         print('training size:', len(train_idx), '  test size:', len(eval_idx))

#         print("Train Date range: {} to {}".format(train_idx.min(),train_idx.max()))
#         print("Valid Date range: {} to {}".format(eval_idx.min(),eval_idx.max()))


#         X, y = df_added_all_scaled.copy(), y_all.copy()
#         # X, y = X_lgb.copy(), y.copy()

#         X_train, X_test = X.loc[train_idx].reset_index(drop=True), X.loc[eval_idx].reset_index(drop=True)
#         y_train, y_test = y.loc[train_idx].reset_index(drop=True), y.loc[eval_idx].reset_index(drop=True)

        
#         lgb = LGBMRegressor(**param_grid)
#         lgb.fit(
#             X_train, 
#             y_train,
#             eval_set = [(X_test, y_test)],
#             eval_metric='rmse',
#             callbacks=[
#                     early_stopping(stopping_rounds=20)
#                 ]
#             )

#         feat_importance["Importance_Fold"+str(i+1)]=lgb.feature_importances_
#         feat_importance.set_index(X_train.columns, inplace=True)

#         y_pred = lgb.predict(X_test)

        
#         # Sharpe Ratio 
#         df_sharpe = pd.DataFrame([])
#         df_sharpe['Date'] = X.loc[eval_idx].index
#         df_sharpe['predict'] = y_pred
#         df_sharpe['Target'] = y_test
#         df_sharpe['Rank'] = (df_sharpe.groupby('Date')['predict'].rank(method='first', ascending=False)-1).astype(int)
#     #         display(X_test.groupby('Date')['Rank'].min())
#     #         display(X_test.groupby('Date')['Rank'].max())
#     #         display(X_test)
#         sharpe = calc_spread_return_sharpe(df_sharpe)
        
#         cv_sharpe.append(sharpe)

#         print(f'Fold {i+1} evaluated in {time.time() - t0 :.3f}s')
#         print(f'Fold {i+1} sharpe ratio score: {sharpe :.6f}')


        
#         assert len(y_test) == len(y_pred)
#         assert y_test.isna().any() == False
        
#         # MSE as objective 
#         cv_score.append(mean_squared_error(y_test.values, y_pred))
        
#     print('cv completed with mse scores:', cv_score)
#     print('cv mean mse score: {:.6f}'.format(np.mean(cv_score)))
#     print('cv mean sharp score: {:.6f}'.format(np.mean(cv_sharpe)))
#     print('cv STD sharp score: {:.6f}'.format(np.std(cv_sharpe)))
#     count += 1

#     feat_importance['avg'] = feat_importance.mean(axis=1)
#     feat_importance = feat_importance.sort_values(by='avg',ascending=True)
#     display(feat_importance)
    
#     return np.mean(cv_score)





In [19]:
# count = 1

# optuna.logging.set_verbosity(optuna.logging.WARNING)

# study = optuna.create_study(direction="minimize", study_name="LGBM Regressor")
# func = lambda trial: objective(trial, df_added, y, train_size=130, test_size=60, fold=8)
# study.optimize(func, n_trials=10)

# joblib.dump(study, '../models/lgbm_fin_10trial_stationary_scaled_train130_test60_fold8.pkl')

In [20]:
# lgb = LGBMRegressor(**maxsr_params, seed=2022)
# lgb.fit(
#     df_added.loc[grid],   #only recent 130 days fit to predict next 60 days
#     y.loc[grid],
#     # eval_set = [(X_test, y_test)],
#     # callbacks=[
#             # early_stopping(stopping_rounds=20)
#         # ]
#     )


# evaluate 3-month  12-06 ~ 3월
# sgrid = supp_data['Date'].unique()[:2]
# print(min(sgrid), max(sgrid))

# biggrid = train_with_supp['Date'].unique()[-300:] #last 300 days
# print(min(biggrid), max(biggrid))

# # train_with_supp
# ts_divsec = divideSecurities(train_with_supp.set_index('Date').loc[biggrid].reset_index())
# feat_supp, _ = add_features_train(ts_divsec)
# X_supp = feat_supp.loc[sgrid] #preprocess_inference(feat_supp.set_index('Date').loc[sgrid].reset_index(), trained_scalers)

# y_test = train_with_supp.set_index('Date').loc[sgrid, ['Target']].reset_index(drop=True)
# y_pred = ensemble.predict(X_supp)

# print('supp data evaluation with r2-score : {:.5f}'.format(r2_score(y_test.values, y_pred)))

# df_sharpe = pd.DataFrame([])
# df_sharpe['Date'] = X_supp.index
# df_sharpe['SecuritiesCode'] = X_supp.SecuritiesCode.reset_index(drop=True)
# df_sharpe['predict'] = y_pred
# df_sharpe['Target'] = y_test
# df_sharpe['Rank'] = (df_sharpe.groupby('Date')['predict'].rank(method='first', ascending=False)-1).astype(int)
# eval_score = calc_spread_return_sharpe(df_sharpe)

# print('sharpe ratio score : {}'.format(eval_score))


### score board
| number | features used | len_train, len_eval, cv | trial | train_score | test_score | remarks |  
| :-----: | :-----: | :-----: | :-----: | :-----: | :-----: | :-----: |
| 1 | rolling(not scaled) | 210, 60, 5 | 50 | R2:-0.001238 SR:0.110312 | R2: -0.01851 SR: -0.182878 |  |
| 2 | rolling(not scaled)- 30 rolling | 130, 60, 8 (also 130 only for test) | 10 | R2:-0.003075 SR:0.127686 | R2: -0.01352 SR: -0.216620 |  |


In [21]:
# # local-api
# sys.path.insert(0, '../input/jpx-local-api')
# from local_api import local_api

# seccodes = np.sort(train['SecuritiesCode'].unique())
# main_list = stock_list.set_index('SecuritiesCode').loc[seccodes, ['33SectorCode']]

# myapi = local_api('../input/jpx-tokyo-stock-exchange-prediction/supplemental_files', 
#                  start_date='2021-12-06', end_date='2021-12-07') # 3-month
# env = myapi.make_env()
# iter_test = env.iter_test()
# for i, (prices, options, financials, trades, secondary_prices, sample_prediction) in enumerate(tqdm(iter_test)):
# #     t0 = time.time()
    
#     # 이전 데이터와 합치고 최근 140일치만 이용한다
#     today = prices.iloc[0]['Date']
#     lastday = str(pd.to_datetime(today) - pd.DateOffset(200))
    
#     # add 33SectorCode
#     prices = prices.set_index('SecuritiesCode').join(main_list).reset_index().astype({
#         'SecuritiesCode':'category',
#         '33SectorCode':'category'})

#     if i == 0:
#         close_df = pd.concat([
#             train_with_supp.loc[
#                 (train_with_supp['Date'] > lastday) & (train_with_supp['Date'] < today)],
#             prices
#             ]).reset_index(drop=True)
#     else:
#         close_df = pd.concat([
#             close_df.loc[
#                 (close_df['Date'] > lastday) & (close_df['Date'] < today)],
#             prices
#             ]).reset_index(drop=True)

    

#     sec_list_infer = divideSecurities(close_df)
#     X, _ = add_features_train(sec_list_infer)
#     # display(X)
#     X = X[X.index == today]
#     # display(X)
    
    
    
#     # X, y
#     X['predict'] = ensemble.predict(X)
#     X['Rank'] = (X['predict'].rank(method='first', ascending=False)-1).astype(int)
#     sample_prediction['Rank'] = X['Rank'].values
#     # display(sample_prediction)
#     # display(myapi.gt_prices)
#     # display(gt_prices)
#     # check Rank
#     assert sample_prediction["Rank"].notna().all()
#     assert sample_prediction["Rank"].min() == 0
#     assert sample_prediction["Rank"].max() == len(sample_prediction["Rank"]) - 1
    
# #     display(sample_prediction)
#     env.predict(sample_prediction)
# print(env.score())

pd.to_datetime 최소한도로 쓸것. 연산 cost가 너무 큼. 특히 inference 단계에서는 쓰지 않기.

In [22]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

In [23]:
seccodes = np.sort(train['SecuritiesCode'].unique())
main_list = stock_list.set_index('SecuritiesCode').loc[seccodes, ['33SectorCode']]

for i, (prices, options, financials, trades, secondary_prices, sample_prediction) in enumerate(iter_test):
#     t0 = time.time()
    
    # 이전 데이터와 합치고 최근 140일치만 이용한다
    today = prices.iloc[0]['Date']
    lastday = str(pd.to_datetime(today) - pd.DateOffset(200))
    
    # add 33SectorCode
    prices = prices.set_index('SecuritiesCode').join(main_list).reset_index().astype({
        'SecuritiesCode':'category',
        '33SectorCode':'category'})

    if i == 0:
        close_df = pd.concat([
            train_with_supp.loc[
                (train_with_supp['Date'] > lastday) & (train_with_supp['Date'] < today)],
            prices
            ]).reset_index(drop=True)
    else:
        close_df = pd.concat([
            close_df.loc[
                (close_df['Date'] > lastday) & (close_df['Date'] < today)],
            prices
            ]).reset_index(drop=True)

    

    sec_list_infer = divideSecurities(close_df)
    X, _ = add_features_train(sec_list_infer)
    # display(X)
    X = X[X.index == today]
    # display(X)
    
    
    
    # X, y
    X['predict'] = ensemble.predict(X)
    X['Rank'] = (X['predict'].rank(method='first', ascending=False)-1).astype(int)
    sample_prediction['Rank'] = X['Rank'].values
    # display(sample_prediction)
    # display(myapi.gt_prices)
    # display(gt_prices)
    # check Rank
    assert sample_prediction["Rank"].notna().all()
    assert sample_prediction["Rank"].min() == 0
    assert sample_prediction["Rank"].max() == len(sample_prediction["Rank"]) - 1
    
#     display(sample_prediction)
    env.predict(sample_prediction)
#     print(time.time() - t0)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


100%|██████████| 2000/2000 [00:41<00:00, 48.04it/s]
